# 手寫辦認

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist = input_data.read_data_sets("MNIST/MNIST_data/",one_hot=True)
print('train image:',mnist.train.images.shape)
print('train label:',mnist.train.labels.shape)
print('test image:',mnist.test.images.shape)
print('test label', mnist.test.labels.shape)

C:\Users\ga456\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
train image: (55000, 784)
train label: (55000, 10)
test image: (10000, 784)
test label (10000, 10)


In [2]:
n_input = 784
n_hidden_1 = 512
n_hidden_2 = 512
n_classes = 10

learning_rate = 0.01
batch_size = 100
total_step = 10000

In [3]:
x = tf.placeholder(tf.float32, [None, n_input])
y_ = tf.placeholder(tf.float32, [None, n_classes])
y_relu_ = tf.placeholder(tf.float32, [None, n_classes])


with tf.name_scope('hidden1'):
    
    weights = tf.Variable(tf.random_normal([n_input, n_hidden_1]),
                            name = 'weights')
    biases = tf.Variable(tf.random_normal([n_hidden_1]),
                            name = 'biases')
    layer_1 = tf.nn.relu(tf.matmul(x, weights) + biases)
    
with tf.name_scope('hidden2'):
    weights = tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]),
                            name = 'weights')
    biases = tf.Variable(tf.random_normal([n_hidden_2]),
                            name = 'biases')
    layer_2 = tf.nn.relu(tf.matmul(layer_1, weights) + biases)

with tf.name_scope('output'):
    weights = tf.Variable(tf.random_normal([n_hidden_2, n_classes]),
                            name = 'weights')
    biases = tf.Variable(tf.random_normal([n_classes]),
                            name = 'biases')
    logits = tf.matmul(layer_2, weights) + biases

In [4]:
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_))
tf.summary.scalar('loss', cross_entropy)

<tf.Tensor 'loss:0' shape=() dtype=string>

In [5]:
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)        

In [6]:
with tf.name_scope('accuracy'):
    y = tf.nn.softmax(logits)
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
tf.summary.scalar('acc',accuracy)

<tf.Tensor 'acc:0' shape=() dtype=string>

In [7]:
init = tf.global_variables_initializer()
merged = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(init)
    
    test_writer = tf.summary.FileWriter('MNIST/logs/106310224_106310232_lr_0.01')
    
    for i in range(total_step):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        sess.run(train_step, feed_dict={x:batch_xs, y_:batch_ys})
                
        #summary, _ = sess.run([merged, train_step], feed_dict={x:batch_xs, y_:batch_ys})
        #train_writer.add_summary(summary,i)
        
        if i%50 == 0:
            summary = sess.run(merged, feed_dict={x:batch_xs, y_:batch_ys})
            test_writer.add_summary(summary,i)
            
        if i %100 == 0:
            loss, acc = sess.run([cross_entropy, accuracy],
                                 feed_dict = {x:batch_xs, y_:batch_ys})
            test_loss, test_acc = sess.run([cross_entropy, accuracy],
                                           feed_dict={x:mnist.test.images, y_:mnist.test.labels})
            
            print('step:{:4d}\t loss:{:.2f}\t acc:{:.2f}\t test_loss{:.2f}\t test_acc:{:.2f}'.
                  format(i, loss, acc, test_loss, test_acc))
    #train_writer.close()
    test_writer.close()

step:   0	 loss:8737.08	 acc:0.25	 test_loss8017.99	 test_acc:0.15
step: 100	 loss:72.16	 acc:0.83	 test_loss81.30	 test_acc:0.84
step: 200	 loss:6.27	 acc:0.97	 test_loss52.88	 test_acc:0.88
step: 300	 loss:31.83	 acc:0.93	 test_loss40.32	 test_acc:0.89
step: 400	 loss:18.32	 acc:0.95	 test_loss34.87	 test_acc:0.90
step: 500	 loss:4.89	 acc:0.97	 test_loss29.54	 test_acc:0.91
step: 600	 loss:12.94	 acc:0.94	 test_loss26.58	 test_acc:0.91
step: 700	 loss:10.11	 acc:0.97	 test_loss24.30	 test_acc:0.92
step: 800	 loss:1.32	 acc:0.97	 test_loss23.48	 test_acc:0.92
step: 900	 loss:9.20	 acc:0.96	 test_loss22.01	 test_acc:0.92
step:1000	 loss:6.61	 acc:0.98	 test_loss21.19	 test_acc:0.92
step:1100	 loss:5.60	 acc:0.98	 test_loss19.51	 test_acc:0.92
step:1200	 loss:0.00	 acc:1.00	 test_loss19.14	 test_acc:0.92
step:1300	 loss:6.22	 acc:0.98	 test_loss18.08	 test_acc:0.92
step:1400	 loss:0.00	 acc:1.00	 test_loss17.58	 test_acc:0.93
step:1500	 loss:2.35	 acc:0.99	 test_loss17.51	 test_acc:0.9